In [1]:
import pandas as pd
import numpy as np
import math

In [7]:
Movies= pd.read_csv("C:\Users\Mayank\Desktop\data_scien\Untitled Folder\Content Based\\movies.csv")
Ratings = pd.read_csv("C:\Users\Mayank\Desktop\data_scien\Untitled Folder\Content Based\\ratings.csv")
Tags= pd.read_csv("C:\Users\Mayank\Desktop\data_scien\Untitled Folder\Content Based\\tags.csv")

In [10]:
# term frequency and inverse document frequency and tf-idf value
TF= Tags.groupby(['movieId','tag'],as_index=False,sort=False).count().rename(columns={'user':'tag_count_TF'})[['movieId','tag','tag_count_TF']]
Tag_distinct = Tags[['tag','movieId']].drop_duplicates()
DF =Tag_distinct.groupby(['tag'], as_index = False, sort = False).count().rename(columns = {'movieId': 'tag_count_DF'})[['tag','tag_count_DF']]
a=math.log10(len(np.unique(Tags['movieId'])))
DF['IDF']=a-np.log10(DF['tag_count_DF'])
TF = pd.merge(TF,DF,on = 'tag', how = 'left', sort = False)
TF['TF-IDF']=TF['tag_count_TF']*TF['IDF']

KeyError: "['tag_count_TF'] not in index"

In [11]:
# unit vector for tf-idf
Vect_len=TF[['movieId','TF-IDF']]
Vect_len['TF-IDF-Sq']=Vect_len['TF-IDF']**2
Vect_len =Vect_len.groupby(['movieId'], as_index = False, sort = False).sum().rename(columns = {'TF-IDF-Sq': 'TF-IDF-Sq-sum'})[['movieId','TF-IDF-Sq-sum']]
Vect_len['vect_len'] = np.sqrt(Vect_len[['TF-IDF-Sq-sum']].sum(axis=1))
TF = pd.merge(TF,Vect_len,on = 'movieId', how = 'left', sort = False)
TF['TAG_WT']=TF['TF-IDF']/TF['vect_len']

NameError: name 'TF' is not defined

In [12]:
#user profile positively with >3.5
Ratings_filter=Ratings[Ratings[‘rating’]>=3.5]
distinct_users=np.unique(Ratings[‘userId’])
user_tag_pref=pd.DataFrame()
i=1
for user in distinct_users[1:2]:
 
 if i%30==0:
 print “user: “, i , “out of: “, len(distinct_users)
 
 user_data= Ratings_filter[Ratings_filter[‘userId’]==user]
 user_data = pd.merge(TF,user_data,on = ‘movieId’, how = ‘inner’, sort = False)

 user_data1 = user_data.groupby([‘tag’], as_index = False, sort = False).sum().rename(columns = {‘TAG_WT’: ‘tag_pref’})[[‘tag’,’tag_pref’]]
 user_data1[‘user’]=user
 user_tag_pref = user_tag_pref.append(user_data1, ignore_index=True)
 i=i+1

SyntaxError: invalid syntax (<ipython-input-12-99294662cd28>, line 2)

In [13]:
distinct_users=np.unique(Ratings_filter[‘userId’])
tag_merge_all=pd.DataFrame()
i=1
for user in distinct_users[1:2]:
 
 user_tag_pref_all= user_tag_pref[user_tag_pref[‘user’]==user]
 distinct_movies = np.unique(TF[‘movieId’])
 j=1
 for movie in distinct_movies:
 
 if j%300==0:
 
 print (“movie: “, j , “out of: “, len(distinct_movies) , “with user: “, i , “out of: “, len(distinct_users))
    
 TF_Movie= TF[TF[‘movieId’]==movie]
 tag_merge = pd.merge(TF_Movie,user_tag_pref_all,on = ‘tag’, how = ‘left’, sort = False)
 tag_merge[‘tag_pref’]=tag_merge[‘tag_pref’].fillna(0)
 tag_merge[‘tag_value’]=tag_merge[‘TAG_WT’]*tag_merge[‘tag_pref’]
 
 TAG_WT_val=np.sqrt(np.sum(np.square(tag_merge[‘TAG_WT’]), axis=0))
 tag_pref_val=np.sqrt(np.sum(np.square(user_tag_pref_all[‘tag_pref’]), axis=0))
 
 tag_merge_final = tag_merge.groupby([‘user’,’movieId’])[[‘tag_value’]].sum().rename(columns = {‘tag_value’: ‘Rating’}).reset_index()
 
 tag_merge_final[‘Rating’]=tag_merge_final[‘Rating’]/(TAG_WT_val*tag_pref_val)

 tag_merge_all = tag_merge_all.append(tag_merge_final, ignore_index=True)
 j=j+1
 i=i+1
tag_merge_all=tag_merge_all.sort_index(by=[‘user’,’Rating’]).reset_index()



SyntaxError: invalid syntax (<ipython-input-13-932c499975b0>, line 1)

In [14]:
#calculate weight sum based on users rating

distinct_users=np.unique(Ratings[‘userId’])
user_tag_pref=pd.DataFrame()
i=1
for user in distinct_users[1:2]:
 
 if i%30==0:
 print “user: “, i , “out of: “, len(distinct_users)
 
 user_data= Ratings[Ratings[‘userId’]==user]
 user_data[‘weight’]=user_data[“rating”]-user_data[“rating”].drop_duplicates().mean()
 user_data1 = pd.merge(TF,user_data,on = ‘movieId’, how = ‘inner’, sort = False)
 user_data1[‘TAG_WT_WTD’] = user_data1[‘TAG_WT’]*user_data1[‘weight’]
 user_data2 = user_data1.groupby([‘tag’], as_index = False, sort = False).sum().rename(columns = {‘TAG_WT_WTD’: ‘tag_pref’})[[‘tag’,’tag_pref’]]
 user_data2[‘user’]=user
 user_tag_pref = user_tag_pref.append(user_data2, ignore_index=True)
 i=i+1


SyntaxError: invalid syntax (<ipython-input-14-ed2026a94e33>, line 3)

In [ ]:
#calculate cosine relation between vectors

distinct_users=np.unique(Ratings[‘userId’])
tag_merge_all=pd.DataFrame()
i=1
for user in distinct_users[1:2]:
 
 user_tag_pref_all= user_tag_pref[user_tag_pref[‘user’]==user]
 distinct_movies = np.unique(TF[‘movieId’])
 j=1
 for movie in distinct_movies:
 
 if j%300==0:
 
 print “movie: “, j , “out of: “, len(distinct_movies) , “with user: “, i , “out of: “, len(distinct_users)
 
 TF_Movie= TF[TF[‘movieId’]==movie]
 tag_merge = pd.merge(TF_Movie,user_tag_pref_all,on = ‘tag’, how = ‘left’, sort = False)
 tag_merge[‘tag_pref’]=tag_merge[‘tag_pref’].fillna(0)
 tag_merge[‘tag_value’]=tag_merge[‘TAG_WT’]*tag_merge[‘tag_pref’]
TAG_WT_val=np.sqrt(np.sum(np.square(tag_merge[‘TAG_WT’]), axis=0))
 tag_pref_val=np.sqrt(np.sum(np.square(user_tag_pref_all[‘tag_pref’]), axis=0))
 
 tag_merge_final = tag_merge.groupby([‘user’,’movieId’])[[‘tag_value’]].sum().rename(columns = {‘tag_value’: ‘Rating’}).reset_index()
 
 tag_merge_final[‘Rating’]=tag_merge_final[‘Rating’]/(TAG_WT_val*tag_pref_val)
 
 tag_merge_all = tag_merge_all.append(tag_merge_final, ignore_index=True)
 j=j+1
 i=i+1
tag_merge_all=tag_merge_all.sort_index(by=[‘user’,’Rating’]).reset_index()
 